In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import h5py

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from matplotlib.pyplot import plot,savefig
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")
from read_count import read_data

In [2]:
col = ["#E64B35CC", "#0072B5CC", "#00A087CC", "#3C5488CC", "#F39B7FCC", "#F7DC05FF", "#FD7446E5",
       "#8491B4CC", "#7E6148CC", "#B09C85CC", "#E18727CC", "#FFDC91E5", "#6A6599E5", "#9467BDB2",
       "#FFFFFFFF", "#0000FFFF", "#FF0000FF", "#00FF00FF", "#000033FF", "#FF00B6FF", "#005300FF", "#FFD300FF",
       "#009FFFFF", "#9A4D42FF", "#00FFBEFF", "#783FC1FF", "#1F9698FF", "#FFACFDFF", "#B1CC71FF", "#F1085CFF",
       "#FE8F42FF", "#DD00FFFF", "#201A01FF", "#720055FF", "#766C95FF", "#02AD24FF", "#C8FF00FF", "#886C00FF",
       "#FFB79FFF", "#858567FF", "#A10300FF", "#14F9FFFF", "#00479EFF", "#DC5E93FF", "#93D4FFFF", "#004CFFFF"]

In [3]:
def plot_cluster(df, data_name, phase, by, y_true, n, ax):
    
    """
        phase: 'split' or 'enhance'. If phase == 'split', using the results after splitting, else using the results after enhancement.
        by: 'clusters' or 'labels'. If by == 'clusters', colored by cluster labels, else colored by true cell types.
        n: n-th dataset in [Human pancreas, Human PBMC, Human kidney, Mouse ES, Mouse hypothalamus, Mouse kidney, Turtle brain]
    """
    
    if phase == 'split':
        umap = umap_init_all[n]
        y_pred = df['Clusters'][0]
    
    else :
        umap = umap_last_all[n]
        y_pred = df['Clusters'][1]
    
    y_pred = np.asarray(y_pred, dtype='int').squeeze()
    K_pred = len(np.unique(y_pred))
    
    ari_pred = np.round(metrics.adjusted_rand_score(y_pred, y_true), 2)
    nmi_pred = np.round(metrics.normalized_mutual_info_score(y_pred, y_true), 2)
    
    if by == 'clusters':
        print('Datasets: {}_{}, ARI={}, NMI={}, k={}'.format(data_name, phase, ari_pred, nmi_pred, K_pred))
        
    adata = sc.AnnData(pd.DataFrame(np.random.rand(len(y_pred), 1)))
    adata.obs['pred'] = y_pred
    adata.obs['pred'] = adata.obs['pred'].astype(str).astype('category')
    adata.obs['true'] = y_true
    adata.obs['true'] = adata.obs['true'].astype(str).astype('category')

    adata.obsm['X_umap'] = umap
    
    if by == 'clusters':
        sc.pl.umap(adata, color=['pred'], ax=ax, show=False, legend_loc='None', size=8)
        if phase == 'split':
            ax.set_title('K={}'.format(K_pred), fontsize=15, family='Arial')
        else:
            ax.set_title('K={} ARI={}'.format(K_pred, ari_pred), fontsize=15, family='Arial')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        
    else:
        sc.pl.umap(adata, color=['true'], ax=ax, show=False, legend_loc='None', palette=col, size=8)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

In [4]:
data_mat = h5py.File('dataset/Human_p.h5')
y_true_human = np.array(data_mat['Y'], dtype='int')
data_mat.close()

data_mat = h5py.File('dataset/Human_.h5')
y_true_pbmc = np.array(data_mat['Y'], dtype='int')
data_mat.close()

data_mat = h5py.File('dataset/Human_k.h5')
y_true_kidney = np.array(data_mat['Y'], dtype='int')
data_mat.close()

mat, obs, var, uns = read_data('dataset/Mouse_E.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true_klein = np.unique(cell_name, return_inverse=True)

mat, obs, var, uns = read_data('dataset/Mouse_h.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true_chen_1 = np.unique(cell_name, return_inverse=True)

adata = sc.AnnData(x)
adata.obs['celltype'] = y_true_chen_1
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_genes=200)
y_true_chen_2 = np.array(adata.obs['celltype']).squeeze()

mat, obs, var, uns = read_data('dataset/Mouse_k.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true_adam = np.unique(cell_name, return_inverse=True)

mat, obs, var, uns = read_data('dataset/Turtle_b.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true_turtle = np.unique(cell_name, return_inverse=True)

In [5]:
fig = plt.figure(figsize=(22, 5))
sub_figs = fig.subfigures(2, 1)
axs = []

for i, sub_fig in enumerate(sub_figs):
    axs.append(sub_fig.subplots(1, 7))
    
axs = np.array(axs)

# Split

In [6]:
human_s = np.load('results/enhancement/scAce_enhance_Seurat_human.npz')
pbmc_s = np.load('results/enhancement/scAce_enhance_Seurat_pbmc.npz')
kidney_s = np.load('results/enhancement/scAce_enhance_Seurat_kidney.npz')
klein_s = np.load('results/enhancement/scAce_enhance_Seurat_klein.npz')
chen_s = np.load('results/enhancement/scAce_enhance_Seurat_chen.npz')
adam_s = np.load('results/enhancement/scAce_enhance_Seurat_adam.npz')
turtle_s = np.load('results/enhancement/scAce_enhance_Seurat_turtle.npz')

In [9]:
datasets = [human_s, pbmc_s, kidney_s, klein_s, chen_s, adam_s, turtle_s]

In [11]:
umap_init_all, umap_last_all = [], []
for data in datasets:
    adata_init = sc.AnnData(data['Embedding'][0])
    sc.pp.neighbors(adata_init)
    sc.tl.umap(adata_init, random_state=0)
    umap_init_all.append(np.array(adata_init.obsm['X_umap']))
    adata_last = sc.AnnData(data['Embedding'][1])
    sc.pp.neighbors(adata_last)
    sc.tl.umap(adata_last, random_state=0)
    umap_last_all.append(np.array(adata_last.obsm['X_umap']))

In [7]:
umap_init_all = np.load("umap/umap_enhance_s_init.npz")['UMAP']
# np.savez("umap/umap_enhance_init.npz", UMAP=umap_init_all)

umap_last_all = np.load("umap/umap_enhance_s_last.npz")['UMAP']
# np.savez("umap/umap_enhance_last.npz", UMAP=umap_last_all)

In [8]:
plot_cluster(human_s, 'Human pancreas', 'split', 'clusters', y_true_human, 0, axs[0][0])
plot_cluster(pbmc_s, 'Human PBMC', 'split', 'clusters', y_true_pbmc, 1, axs[0][1])
plot_cluster(kidney_s, 'Human kidney', 'split', 'clusters', y_true_kidney, 2, axs[0][2])
plot_cluster(klein_s, 'Mouse ES', 'split', 'clusters', y_true_klein, 3, axs[0][3])
plot_cluster(chen_s, 'Mouse kidney', 'split', 'clusters', y_true_chen_2, 4, axs[0][4])
plot_cluster(adam_s, 'Mouse hypothalamus', 'split', 'clusters', y_true_adam, 5, axs[0][5])
plot_cluster(turtle_s, 'Turtle brain', 'split', 'clusters', y_true_turtle, 6, axs[0][6])

Datasets: Human pancreas_split, ARI=0.57, NMI=0.79, k=18
Datasets: Human PBMC_split, ARI=0.59, NMI=0.72, k=13
Datasets: Human kidney_split, ARI=0.4, NMI=0.66, k=24
Datasets: Mouse ES_split, ARI=0.52, NMI=0.73, k=14
Datasets: Mouse kidney_split, ARI=0.88, NMI=0.8, k=19
Datasets: Mouse hypothalamus_split, ARI=0.64, NMI=0.78, k=14
Datasets: Turtle brain_split, ARI=0.56, NMI=0.74, k=20


In [9]:
fig

<Figure size 2200x500 with 14 Axes>

In [10]:
plot_cluster(human_s, 'Human pancreas', 'split', 'labels', y_true_human, 0, axs[1][0])
plot_cluster(pbmc_s, 'Human PBMC', 'split', 'labels', y_true_pbmc, 1, axs[1][1])
plot_cluster(kidney_s, 'Human kidney', 'split', 'labels', y_true_kidney, 2, axs[1][2])
plot_cluster(klein_s, 'Mouse ES', 'split', 'labels', y_true_klein, 3, axs[1][3])
plot_cluster(chen_s, 'Mouse kidney', 'split', 'labels', y_true_chen_2, 4, axs[1][4])
plot_cluster(adam_s, 'Mouse hypothalamus', 'split', 'labels', y_true_adam, 5, axs[1][5])
plot_cluster(turtle_s, 'Turtle brain', 'split', 'labels', y_true_turtle, 6, axs[1][6])

In [11]:
fig

<Figure size 2200x500 with 14 Axes>

In [12]:
plt.savefig('Figures/Figure7B.svg', dpi=300, format='svg', bbox_inches='tight')

# Enhance

In [13]:
fig = plt.figure(figsize=(22, 5))
sub_figs = fig.subfigures(2, 1)
axs = []

for i, sub_fig in enumerate(sub_figs):
    axs.append(sub_fig.subplots(1, 7))

axs = np.array(axs)

In [14]:
plot_cluster(human_s, 'Human pancreas', 'enhance', 'clusters', y_true_human, 0, axs[0][0])
plot_cluster(pbmc_s, 'Human PBMC', 'enhance', 'clusters', y_true_pbmc, 1, axs[0][1])
plot_cluster(kidney_s, 'Human kidney', 'enhance', 'clusters', y_true_kidney, 2, axs[0][2])
plot_cluster(klein_s, 'Mouse ES', 'enhance', 'clusters', y_true_klein, 3, axs[0][3])
plot_cluster(chen_s, 'Mouse kidney', 'enhance', 'clusters', y_true_chen_2, 4, axs[0][4])
plot_cluster(adam_s, 'Mouse hypothalamus', 'enhance', 'clusters', y_true_adam, 5, axs[0][5])
plot_cluster(turtle_s, 'Turtle brain', 'enhance', 'clusters', y_true_turtle, 6, axs[0][6])

Datasets: Human pancreas_enhance, ARI=0.9, NMI=0.88, k=5
Datasets: Human PBMC_enhance, ARI=0.81, NMI=0.81, k=8
Datasets: Human kidney_enhance, ARI=0.53, NMI=0.72, k=12
Datasets: Mouse ES_enhance, ARI=0.91, NMI=0.92, k=5
Datasets: Mouse kidney_enhance, ARI=0.79, NMI=0.79, k=8
Datasets: Mouse hypothalamus_enhance, ARI=0.9, NMI=0.89, k=8
Datasets: Turtle brain_enhance, ARI=0.72, NMI=0.72, k=5


In [15]:
plot_cluster(human_s, 'Human pancreas', 'enhance', 'labels', y_true_human, 0, axs[1][0])
plot_cluster(pbmc_s, 'Human PBMC', 'enhance', 'labels', y_true_pbmc, 1, axs[1][1])
plot_cluster(kidney_s, 'Human kidney', 'enhance', 'labels', y_true_kidney, 2, axs[1][2])
plot_cluster(klein_s, 'Mouse ES', 'enhance', 'labels', y_true_klein, 3, axs[1][3])
plot_cluster(chen_s, 'Mouse kidney', 'enhance', 'labels', y_true_chen_2, 4, axs[1][4])
plot_cluster(adam_s, 'Mouse hypothalamus', 'enhance', 'labels', y_true_adam, 5, axs[1][5])
plot_cluster(turtle_s, 'Turtle brain', 'enhance', 'labels', y_true_turtle, 6, axs[1][6])

In [16]:
fig

<Figure size 2200x500 with 14 Axes>

In [17]:
plt.savefig('Figures/Figure7C.svg', dpi=300, format='svg', bbox_inches='tight')